In [ ]:
#Necessary Libraries for plotting charts
#plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)

In [ ]:
#Function to return pie plot to display ratings distribution in reviews of a company
def pieplotratingsdistribution(df,CompanyName):
    labels = df[df['CompanyName']==CompanyName]['OverallRating'].value_counts().index
    values = df[df['CompanyName']==CompanyName]['OverallRating'].value_counts().values
    trace = go.Pie(labels=labels, values=values)
    py.iplot([trace], filename='basic_pie_chart')

In [ ]:
#Code to plot Stacked bar chart
def stackedbarplot(df):
    company_names=list(df['CompanyName'].unique())
    Employee_categories=list(df['AuthorEmploymentStatus'].unique())
    count_list=[]
    for j in Employee_categories:
        for i in company_names:
            count_list.append(df[(df['CompanyName']==i)&(df['AuthorEmploymentStatus']==j)].shape[0])
    temp=[]
    for i in range(0, len(count_list),len(company_names)):
        temp.append(list(count_list[i:i + len(company_names)]))
    data=[] 
    k=0
    for i in Employee_categories:
        data.append(go.Bar(name=i,x=company_names,y=temp[k]))
        k=k+1
    fig = go.Figure(data=data)
# Change the bar mode
    fig.update_layout(barmode='stack')
    fig.show()

In [ ]:
#Plotting Word Count distribution bar plot
def plotwordcountdistribution(df,reviewtype):
    #reviewtyepe is 'Pros_Modified_Text','Cons_Modified_Text' in our case
    def comment_len(x):
        if type(x) is str:
            return len(x.split())
        else:
            return 0
    df['review_len'] = df[reviewtype].apply(comment_len)
    length_scale=[0,5,15,25,50,75,100,200,10000]#Change length scale according to your requirement
    out = pd.cut(df['review_len'],length_scale)
    ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(12,8))
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x() * 1.040, p.get_height() * 1.015))
    plt.show()
    df.drop(columns=['review_len'])

In [ ]:
#Sentiment score anamolies
def sentianamolybarplot(df):
    polarity_scale=[0.0,0.2,0.4,0.6,0.8,1]
    #'Review_polarity' is column name of sentiment score calculated for whole review.
    df2=df[(df['Review_polarity']>0)&(df['OverallRating']<=3)]
    out = pd.cut(df2['Review_polarity'],polarity_scale)
    ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(12,8))
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x() * 1.040, p.get_height() * 1.015))
    plt.show()

In [ ]:
#Distribution of polarity score
def polarityscorehistplot(df,companyname):
    df['Review_polarity'] = df['Pros'].map(lambda text: TextBlob(text).sentiment.polarity)
    fig = px.histogram(df, x="'Review_polarity'")
    fig.update_layout(title_text='Distribution of sentiment polarity in Reviews', template="plotly_white")
    fig.show()

In [ ]:
#Bar plot of top n bi-grams
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def bar_plot_topbigrams(CompanyName,ReviewType):
    common_words = get_top_n_bigram(df[df['CompanyName']==CompanyName][ReviewType], 20)
    df1 = pd.DataFrame(common_words, columns = ['word' , 'count'])
    fig = px.bar(df1, x='word', y='count')
    fig.update_layout(title_text=ReviewType[9:13]+' Review Bi-gram count top 20 for '+CompanyName, template="plotly_white")
    return fig

In [ ]:
#Bar Plot of top n trigrams
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def bar_plot_toptrigrams(CompanyName,ReviewType):
    common_words = get_top_n_trigram(df[df['CompanyName']==CompanyName][ReviewType], 20)
    df1 = pd.DataFrame(common_words, columns = ['word' , 'count'])
    fig = px.bar(df1, x='word', y='count')
    fig.update_layout(title_text=ReviewType[9:13]+' Review Tri-gram count top 20 for '+CompanyName, template="plotly_white")
    return fig